# Data Acquisition (ETL) - test

**Starting Point:**  
The article data originates from a Kafka datastream. It is not normalized (so it cannot be analyzed directly) and requires Active Directory login (making collaboration difficult).  
- [View Kafka topic data (AKHQ)](https://akhq.pdp.production.admin.srgssr.ch/ui/strimzi/topic/articles-v2/data?sort=NEWEST&partition=All)

- **Processing steps:**  
  1. Read article data from the Delta table populated from Kafka.
  2. Flatten and transform nested fields (e.g., titles, resources, contributors) using a SQL view.
  3. Create a Spark DataFrame from the flattened view and inspect the results.
  4. Write the DataFrame to a Delta table for analytics and automation.
  5. Export a <25MB Parquet sample with only public data for sharing (e.g., via GitHub).

**Goal:**  
The data should be available as a Parquet file for sharing. Since the dataset is large (5GB), only a public sample is exported for easy distribution.

**Access Control:**  
To guarantee data integrity and protect sensitive information, data distribution is based on user access rights. Entitled users can access the full confidential dataset, while restricted users are provided with only the public sample. This ensures that only authorized users can view sensitive data, maintaining compliance and data security.

## Step-01: Read from Kafka (SQL)

The following steps read article data from the Delta table `udp_prd_atomic.pdp.articles_v2`, which is assumed to be populated from a Kafka stream. The original Kafka data contains nested lists and complex structures (e.g., for multilingual fields or arrays of resources). In the transformation, the SQL view `articles_flat` flattens this nested data by extracting relevant fields and, where multiple values exist (such as for titles in different languages), selects the first available entry—typically prioritizing German (`'de'`) or otherwise the first value. This process prepares the data, along with Kafka metadata (key, topic, partition, offset, timestamp), for further analysis in a flat, tabular format.

## Step-02: Create DataFrame and Visually Inspect Results

The code below runs a Spark SQL query against the temporary view `articles_flat`, loads the result into a Spark DataFrame named `df`, and then displays the DataFrame for visual inspection. This step materializes the flattened article data so it can be further processed or written to a Delta table.

## Step-03: Save Data to Delta Table

In the next steps, the data will be saved both to a Delta table (for better automation and analytics).

### Write to (private) Delta Table - all articles

The following code appends the transformed DataFrame `df` to the Delta table `swi_audience_prd.pdp_articles_v2.articles_v2`. It writes in **append** mode, uses the **Delta** format, and enables **schema merging** so that any new columns are automatically added to the target table without overwriting existing data.

- Contains all articles (**confidential**)


### Write to (public) Parquet File - selected articles, manually Upload to GitHub

Export a <25 MB sample of the data with only public data as a Parquet file for easy sharing via GitHub.  
**Note:** The Parquet file must be manually downloaded from Databricks and then uploaded to your GitHub repository.

...now manually:

1. **Open the CSV file in Databricks:**
   - Navigate to [Databricks Volume Browser](https://adb-4119964566130471.11.azuredatabricks.net/explore/data/volumes/swi_audience_prd/pdp_articles_v2/pdp_articles_v2_volume?o=4119964566130471) in the Databricks workspace file browser.

2. **Download the file:**
   - Right-click on `export_articles_v2_sample25mb.parquet` and select **"Download"** to save the file to your local machine.

3. **Upload the file to GitHub:**
   - Go to [GitHub Folder](https://github.com/Tao-Pi/CAS-Applied-Data-Science/tree/main/Module-3/01_Module%20Final%20Assignment).
   - Click **"Add file"** > **"Upload files"**.
   - Drag and drop `export_articles_v2_sample25mb.parquet` or use the file picker to select it.
   - Commit the changes to upload the file.


## Step-04: Load Data Based on User Rights

The next step is to load the data, with access determined by user rights:

- **Restricted users** can load only the public data sample (e.g., the Parquet file exported for sharing).
- **Entitled users** can load the full, confidential dataset from the Delta table.

This ensures that sensitive information is only accessible to authorized users, while still allowing broader access to public data for collaboration and analysis.

In [0]:
%pip install pandas
%pip install pyarrow
%pip install fastparquet
#%pip install -U sentence-transformers torch safetensors accelerate


In [0]:
import pandas as pd
url = "https://github.com/Tao-Pi/CAS-Applied-Data-Science/raw/main/Module-3/01_Module%20Final%20Assignment/export_articles_v2_sample25mb.parquet"
srgssr_article_corpus = pd.read_parquet(url, engine="fastparquet")

In [0]:
has_read_access_udp_articles_v2 = False

# Dataset Overview

In this chapter, we provide a brief overview of the dataset used for analysis. We indicate whether the loaded dataset is the full confidential version or the public sample, report the total number of articles available, and present a first look at the articles data to understand its structure and content.


## Step 1: Check Dataset Version (Confidential vs Public)

Here we check and indicate whether the loaded dataset is the full confidential version or the public sample.

In [0]:
def format_rowcount(n):
    if n >= 1_000_000:
        return f"more than {n // 1_000_000} million"
    elif n >= 1_000:
        return f"more than {n // 1_000} thousand"
        return f"more than {n // 1_000_000} Mio."
    elif n >= 1_000:
        return f"more than {n // 1_000} Tsd."
    else:
        return f"{n}"

if has_read_access_udp_articles_v2:
    rowcount = srgssr_article_corpus.count()
    print(f"congrats: you have successfully read the full data set. This contains the full corpus of {format_rowcount(rowcount)} Articles published by SRG-SSR as plain text together with some relevant metadata. You can access the dataframe object by calling 'srgssr_article_corpus' from Python now.")
else:
    if isinstance(srgssr_article_corpus, pd.DataFrame):
        rowcount = len(srgssr_article_corpus)
    else:
        rowcount = srgssr_article_corpus.count()
    print(f"congrats: you have successfully read the publically available (sampled) data set. This contains an excerpt of {format_rowcount(rowcount)} articles within SRG-SSR as plain text together with some relevant metadata. You can access the dataframe object by calling 'srgssr_article_corpus' from Python now.")


## Step 2: Overview of the Data

In this step, we provide an overview of the data contained in the loaded dataset. This includes a summary of the available articles and a first look at their structure and content.

In [0]:
# Falls DataFrame leer ist → leeres dict
first_row = srgssr_article_corpus.iloc[0].to_dict() if not srgssr_article_corpus.empty else {}

cols_info = [
    {
        "column": col,
        "type": str(dtype),
        "example": first_row.get(col, None)
    }
    for col, dtype in srgssr_article_corpus.dtypes.items()
]

# Schön anzeigen
import pandas as pd
pd.DataFrame(cols_info).head(20)


## Step 3: A Closer Look

In this step, we take a deeper look at the loaded dataset, exploring its structure and content in more detail.

In [0]:
display(srgssr_article_corpus)

#Analyses
This is where analyses are performed. This is work in progress. Some ideas:

**Story 1:** I want to quickly search all existing articles without the need to use Google. I want to do that because when I write a story, I want to make sure the same story was not just written by my colleagues working in a different branch.

**Story 2:** I want to find out what topics SRG writes about – this could, for example, be used for navigation (News / Sport / etc.).

**Story 3:** I want to translate all existing articles into all languages used. This way, I can multiply the offer easily. Instead of having some articles in French and some in English, I will have all articles available in all of our 11 languages.

*List other ideas here...*

In [0]:
srgssr_article_corpus = srgssr_article_corpus.head(1000)

## USE CASE: Quickly Search All Existing Articles

I want to quickly search all existing articles without the need to use Google. I want to do that because when I write a story, I want to make sure the same story was not just written by my colleagues working in a different branch.

**Approach:**
- Implement a semantic search feature within Databricks that allows users to search articles by keywords, phrases, or topics.
- Use text embeddings (e.g., with Sentence Transformers) to represent article content and enable similarity-based search.
- Provide a simple search interface where users can enter queries and retrieve the most relevant articles.
- Optionally, add filters for date, author, or branch to refine search results.

In [0]:
%pip install sentence-transformers

In [0]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer

TEXT_COL = "content_text_csv"
ID_COL = "id"

df = srgssr_article_corpus.copy()
df[TEXT_COL] = df[TEXT_COL].fillna("").astype(str)

_model = None
def get_embedder():
    global _model
    if _model is None:
        _model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
    return _model

model = get_embedder()
emb_matrix = model.encode(
    df[TEXT_COL].tolist(),
    batch_size=64,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True,
)

ids = df[ID_COL].tolist()
texts = df[TEXT_COL].tolist()

def semantic_search(query: str, top_k: int = 10) -> pd.DataFrame:
    q = model.encode([query], convert_to_numpy=True, normalize_embeddings=True)[0]
    sims = emb_matrix @ q
    top_idx = np.argpartition(-sims, kth=min(top_k, len(sims)-1))[:top_k]
    top_idx = top_idx[np.argsort(-sims[top_idx])]
    return pd.DataFrame({
        "id": [ids[i] for i in top_idx],
        "content_text_csv": [texts[i] for i in top_idx],
        "similarity": [float(sims[i]) for i in top_idx],
    })

# Example usage:
results = semantic_search("climate change", top_k=10)
display(results)


##USE CASE: find out what topics SRG writes about.
**Approach:**  
- Read the text from the `content_text_csv` column of the articles.
- Compute similarity between article contents, e.g., by embedding the texts and using a Random Forest or other clustering/classification methods to group similar articles.
- Identify clusters of similar content to reveal common topics or themes.
- Use these clusters to enhance navigation and filtering options for users.

In [0]:
from sklearn.cluster import KMeans
import pandas as pd

# Use existing embeddings from previous cell
n_clusters = 10  # Adjust as needed

kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init="auto")
labels = kmeans.fit_predict(emb_matrix)

df_clusters = pd.DataFrame({
    "id": ids,
    "content_text_csv": texts,
    "cluster": labels
})

display(df_clusters)


## USE CASE: Translate All Existing Articles into All Languages Used

Goal: Automatically translate every article into all supported languages, so each article is available in every language used by SRG.

**Approach:**
- For each article, use the `ai_translate` function to generate translations for all target languages.
- Store the translated articles alongside the originals for easy access and analytics.

**Example (SQL):**
sql
SELECT
  *,
  ai_translate(title, 'fr') AS title_fr,
  ai_translate(title, 'it') AS title_it,
  ai_translate(title, 'en') AS title_en,
  ai_translate(title, 'rm') AS title_rm
FROM articles_flat

*(Repeat for all relevant text fields and all required languages.)*

In [0]:
# List of target languages and their codes (Databricks ai_translate supports: en, fr, it, es, de, pt, zh, ja, ru, ar)
lang_map = {
    "en": "en",   # English
    "fr": "fr",   # French
    "it": "it",   # Italian
    "spa": "es",  # Spanish
    "ger": "de",  # German
    "chi": "zh",  # Chinese
    "jap": "ja",  # Japanese
    "rus": "ru",  # Russian
    "ar": "ar",   # Arabic
    "por": "pt"   # Portuguese
}

# Convert pandas DataFrame to Spark DataFrame if needed
if isinstance(srgssr_article_corpus, pd.DataFrame):
    srgssr_article_corpus_spark = spark.createDataFrame(srgssr_article_corpus)
else:
    srgssr_article_corpus_spark = srgssr_article_corpus

from pyspark.sql.functions import expr

# Add translation columns using ai_translate for each language
df_translated = srgssr_article_corpus_spark
for lang, db_lang in lang_map.items():
    df_translated = df_translated.withColumn(
        f"content_text_{lang}",
        expr(f"ai_translate(content_text_csv, '{db_lang}')")
    )

display(df_translated)

#Hints and Notes
> **Hint:**  
> For a temporary Jupyter environment to experiment or explore data, consider using [RenkuLab](https://renkulab.io/p/snsf-anoxia-project/proxy-proxy/sessions/01JX2TG1RZ9J0PQ53H3RT81BD4/start). RenkuLab offers cloud-based notebook sessions—no local setup required.

> **Note:**  
> RenkuLab sessions may require authentication and have limited resources. Save your work frequently, as sessions can time out or be terminated after inactivity.